In [7]:
using Gridap
using GridapMakie, CairoMakie, FileIO
using Gridap.FESpaces
using Gridap.ReferenceFEs
using Gridap.Arrays
using Gridap.Algebra
using Gridap.Geometry
using Gridap.Fields
using Gridap.CellData
using FillArrays
using Test
using InteractiveUtils
using PyPlot
using JSON3
using Dates
# using Interpolations

In [8]:
include("Julia_functions/indicator_chi.jl")
include("Julia_functions/cost_fun.jl")
include("Julia_functions/gradient_descent.jl")
include("Julia_functions/solvers.jl")
include("Julia_functions/find.jl")
include("Julia_functions/res_plot.jl")
include("Julia_functions/get_domain.jl")
include("Julia_functions/get_price_temp.jl")
include("Julia_functions/post_processing.jl")

get_control_int (generic function with 1 method)

In [ ]:
#Skriv kode for å manufactured solution

T(x,t) = sin(x[1] + x[2] + t)

print(T((1,1),1))

T_t(x,t) = cos(x[1]+x[2]+t)
ΔT(x,t) = -2*sin(x[1]+x[2]+t)
Q(x,t) = T_t - ΔT

T_out_up(x,t) = cos(x[1]+x[2]+t) - T(x,t)
T_out_down(x,t) = -cos(x[1]+x[2]+t) - T(x,y)
T_out_left(x,t) = -cos(x[1]+x[2]+t)- T(x,y)
T_out_right(x,t) = cos(x[1]+x[2]+t)- T(x,y)

ls = LUSolver()
θ = 0.5
Δt = 0.05
solver = ThetaMethod(ls, Δt, θ)
h = 1.0
k = 1
ρ = 1
c = 1
t0 = 0
tF = 1
TIni(x) = 1



domain = (0, 1, 0, 1)
partition = (25, 25)
model = CartesianDiscreteModel(domain, partition)

#???????
labels = get_face_labeling(model)
add_tag_from_tags!(labels, "up", x -> isapprox(x[2], 1.0; atol=1e-8))
add_tag_from_tags!(labels, "down", x -> isapprox(x[2], 0.0; atol=1e-8))
add_tag_from_tags!(labels, "left", x -> isapprox(x[1], 0.0; atol=1e-8))
add_tag_from_tags!(labels, "right", x -> isapprox(x[1], 1.0; atol=1e-8))


neumanntags = ["up", "down", "left","right"]
Γ = BoundaryTriangulation(model,tags=neumanntags)
dΓ = Measure(Γ,degree)

order = 1
degree = 2*order

Ω = Triangulation(model)
dΩ = Measure(Ω, degree)

Γ_up = BoundaryTriangulation(model, tags="up")
dΓ_up = Measure(Γ_up, degree)

Γ_down = BoundaryTriangulation(model, tags="down")
dΓ_down = Measure(Γ_down, degree)

Γ_left = BoundaryTriangulation(model, tags="left")
dΓ_left = Measure(Γ_left, degree)

Γ_right = BoundaryTriangulation(model, tags="right")
dΓ_right = Measure(Γ_right, degree)


reffe = ReferenceFE(lagrangian,Float64,order)
Testspace = TestFESpace(model,reffe,conformity=:H1)                         
Trialspace = TransientTrialFESpace(Testspace)                                
Uspace = FESpace(model, reffe, conformity=:H1)

a_dt(t, dtT, ϕ) = ∫(c*dtT*ϕ*ρ)dΩ

a_SE(t, T, ϕ) =
    ∫(k * ∇(T) ⋅ ∇(ϕ))dΩ +
    ∫(h * T * ϕ)dΓ

l_SE(t, ϕ) =
    ∫(Qt(t) * ϕ)dΩ +
    ∫(T_out_up(x,t) * ϕ * h_wall)dΓ_up +
    ∫(T_out_down(x,t) * ϕ * h_wall)dΓ_down +
    ∫(T_out_left(x,t) * ϕ * h_wall)dΓ_left +
    ∫(T_out_right(x,t) * ϕ * h_wall)dΓ_right +

op_SE = TransientLinearFEOperator((a_dt, a_SE), l_SE, Trialspace, Testspace, constant_forms=(true, true))

T = solve(solver, op_SE, t0, tF, TIni)



0.1411200080598672

UndefVarError: UndefVarError: `add_tag_from_function!` not defined in `Main`
Suggestion: check for spelling errors or missing imports.